<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-27 14:30:58
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.37 C
-------------------
Today PnL: -53.21 K (-0.39%)
Current PnL: -36.96 L (-23.1%)
CY Booked + Current PnL: -21.28 L (-13.3%)
-------------------
Total profit:  1.42 L
Total loss:  -38.38 L
-------------------
Total Booked + Current PnL: 5.70 L (4.29%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.41%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.04 C (75.33%)
Deployed:  1.33 C
Current:  1.37 C
CAGR/XIRR %: 1.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,0.06,6.66,10.32,17.67,16562.0,10028.0,160488.0,478.51,29.12,58.0,L-LC,11.08,185.0,0.61,1.19,28.83,XY25,ATH,MINING
50,MASFIN,-3.95,-0.34,21.98,21.57,21463.0,-330.0,97650.0,397.04,-14.18,48.0,H-SC,6.58,167.0,-0.02,0.72,42.50,XR,ATH,FINANCE
77,TTKPRESTIG,-0.69,-31.94,47.07,0.09,32283.0,-32192.0,68585.0,770.00,59.23,31.0,M-SC,11.60,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
51,MEDANTA,-1.30,-4.26,29.60,24.08,35976.0,-5410.0,121540.0,1486.00,-4.88,51.0,X-SC,7.96,87.0,-0.15,0.90,11.32,XY24,NTT,HEALTHCARE
37,ICICIGI,-2.23,0.80,18.39,19.34,37096.0,1609.0,201718.0,2252.93,-15.18,49.0,X-MC,1.09,66.0,0.04,1.49,16.87,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,7.47,-28.33,71.66,23.03,91973.0,-50724.0,128347.0,216.00,40.35,45.0,H-SC,5.24,166.0,-0.55,0.95,27.35,XR,NTT,BANKS
71,TANLA,5.99,-47.00,314.48,119.69,451323.0,-127247.0,143514.0,1943.92,-46.61,47.0,H-SC,19.85,147.0,-0.28,1.06,13.38,AR,ATH,IT
27,FINCABLES,5.15,16.31,78.75,107.91,135238.0,24087.0,171731.0,1641.55,4.52,75.0,M-SC,7.07,220.0,0.18,1.27,29.20,OX40N,ATH,CABLES
24,DIXON,4.33,-26.83,79.38,31.26,134044.0,-61904.0,168864.0,18931.72,-67.33,41.0,X-MC,24.10,54.0,-0.46,1.25,4.33,X40N,ATH,IT
65,SIEMENS,2.24,-8.34,36.93,25.51,62995.0,-15515.0,170580.0,4671.50,43.36,66.0,H-LC,3.55,51.0,-0.25,1.26,25.70,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-4.42,-90.35,1187.35,24.20,349675.0,-275800.0,29450.0,3033.00,-85.87,36.0,X-SC,2.45,97.0,-0.79,0.22,23.34,X40N,NTT,FINANCE
50,MASFIN,-3.95,-0.34,21.98,21.57,21463.0,-330.0,97650.0,397.04,-14.18,48.0,H-SC,6.58,167.0,-0.02,0.72,42.50,XR,ATH,FINANCE
17,CAMS,-3.68,-10.57,39.18,24.48,88249.0,-26615.0,225241.0,950.00,-14.36,35.0,X-SC,5.66,89.0,-0.30,1.67,10.67,X40N,NTT,MISC
55,RAJESHEXPO,-3.22,-71.77,254.48,0.07,98545.0,-98453.0,38724.0,518.00,1319.19,31.0,L-SC,9.91,268.0,-1.00,0.29,0.00,OX40N,NTT,JEWELLERY
38,ICICIPRULI,-2.91,8.37,20.63,30.73,44044.0,16492.0,213497.0,790.00,-20.69,51.0,X-MC,4.14,78.0,0.37,1.58,22.21,X40,ATH,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-3.95,-0.34,21.98,21.57,21463.0,-330.0,97650.0,397.04,-14.18,48.0,H-SC,6.58,167.0,-0.02,0.72,42.5,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,5.15,16.31,78.75,107.91,135238.0,24087.0,171731.0,1641.55,4.52,75.0,M-SC,7.07,220.0,0.18,1.27,29.20,OX40N,ATH,CABLES
66,SIS,0.58,-30.26,73.88,21.26,57206.0,-33601.0,77431.0,528.00,1818.65,40.0,H-SC,5.56,168.0,-0.59,0.57,6.75,OX40N,NTT,MISC
77,TTKPRESTIG,-0.69,-31.94,47.07,0.09,32283.0,-32192.0,68585.0,770.00,59.23,31.0,M-SC,11.60,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
48,KANSAINER,1.67,-32.00,66.86,13.47,122611.0,-86283.0,183384.0,340.00,-41.65,39.0,H-SC,12.99,162.0,-0.70,1.36,3.09,XY24,NTT,PAINTS
41,INDIGOPNTS,-0.99,-33.36,50.09,0.02,58267.0,-58235.0,116324.0,1408.00,77.46,26.0,M-SC,12.19,222.0,-1.00,0.86,1.18,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1.76,22.95,18.09,45.2,42648.0,44013.0,235755.0,1856.0,40.79,66.0,X-MC,9.12,79.0,1.03,1.75,31.31,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,0.06,6.66,10.32,17.67,16562.0,10028.0,160488.0,478.51,29.12,58.0,L-LC,11.08,185.0,0.61,1.19,28.83,XY25,ATH,MINING
27,FINCABLES,5.15,16.31,78.75,107.91,135238.0,24087.0,171731.0,1641.55,4.52,75.0,M-SC,7.07,220.0,0.18,1.27,29.20,OX40N,ATH,CABLES
1,ABB,-0.86,11.55,30.49,45.57,88982.0,30221.0,291840.0,7934.00,-28.35,66.0,H-MC,8.81,125.0,0.34,2.16,29.60,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.86,11.55,30.49,45.57,88982.0,30221.0,291840.0,7934.00,-28.35,66.0,H-MC,8.81,125.0,0.34,2.16,29.60,AR,NTT,ELECTRICAL
27,FINCABLES,5.15,16.31,78.75,107.91,135238.0,24087.0,171731.0,1641.55,4.52,75.0,M-SC,7.07,220.0,0.18,1.27,29.20,OX40N,ATH,CABLES
50,MASFIN,-3.95,-0.34,21.98,21.57,21463.0,-330.0,97650.0,397.04,-14.18,48.0,H-SC,6.58,167.0,-0.02,0.72,42.50,XR,ATH,FINANCE
85,ZYDUSLIFE,-1.43,-2.45,39.04,35.63,79812.0,-5138.0,204436.0,1286.17,-15.32,56.0,H-MC,4.88,120.0,-0.06,1.51,13.81,AR,ATH,PHARMA
40,INDIAMART,-1.16,-7.38,123.19,106.72,140727.0,-9100.0,114236.0,4810.62,-58.20,42.0,H-SC,8.08,140.0,-0.06,0.85,14.28,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.12,-38.69,156.37,57.19,312682.0,-126164.0,199963.0,485.00,-64.88,18.0,X-SC,18.83,81.0,-0.40,1.48,0.00,XY24,NTT,FMCG
43,INFY,1.16,-11.23,51.23,34.24,142292.0,-35136.0,277752.0,1972.00,-29.97,25.0,X-LC,8.69,3.0,-0.25,2.06,2.23,X40,NTT,IT
12,BATAINDIA,-0.60,-49.05,165.37,35.20,108413.0,-63112.0,65558.0,2096.00,-14.08,28.0,X-SC,22.33,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
34,HCLTECH,1.17,-9.83,37.33,23.83,81436.0,-23772.0,218152.0,1908.19,-5.29,30.0,X-LC,6.06,8.0,-0.29,1.62,4.66,X40,ATH,IT
53,PGHH,-0.03,-14.20,53.86,32.01,98672.0,-30320.0,183200.0,17616.87,-42.03,31.0,X-MC,9.58,57.0,-0.31,1.36,0.06,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIGI,-2.23,0.80,18.39,19.34,37096.0,1609.0,201718.0,2252.93,-15.18,49.0,X-MC,1.09,66.0,0.04,1.49,16.87,X40,ATH,INSURANCE
78,UNITDSPR,-0.06,1.96,18.44,20.77,44023.0,4592.0,238736.0,1644.00,-9.55,50.0,X-MC,1.29,64.0,0.10,1.77,9.51,X40N,NTT,BREWERIES
22,DABUR,-0.96,1.55,41.51,43.70,104567.0,3841.0,251909.0,735.00,-7.83,55.0,X-MC,1.48,70.0,0.04,1.87,16.83,XY24,BTT,FMCG
59,RELIANCE,-0.12,-9.81,38.92,25.30,70546.0,-19705.0,181258.0,1952.00,-11.96,42.0,X-LC,2.29,21.0,-0.28,1.34,21.41,XY25,BTT,REFINERIES
5,ANGELONE,-4.42,-90.35,1187.35,24.20,349675.0,-275800.0,29450.0,3033.00,-85.87,36.0,X-SC,2.45,97.0,-0.79,0.22,23.34,X40N,NTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,-1.20,-33.02,145.07,64.15,231213.0,-78571.0,159380.0,3906.00,-49.40,33.0,X-SC,9.50,85.0,-0.34,1.18,0.0,XY24,BTT,CEMENT
12,BATAINDIA,-0.60,-49.05,165.37,35.20,108413.0,-63112.0,65558.0,2096.00,-14.08,28.0,X-SC,22.33,91.0,-0.58,0.49,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,-0.71,-25.52,108.74,55.47,197041.0,-62082.0,181204.0,181.50,-36.04,32.0,X-MC,17.21,60.0,-0.32,1.34,0.0,X40N,ATH,FINANCE
8,AWL,-0.12,-38.69,156.37,57.19,312682.0,-126164.0,199963.0,485.00,-64.88,18.0,X-SC,18.83,81.0,-0.40,1.48,0.0,XY24,NTT,FMCG
52,PAGEIND,-0.50,-18.77,60.21,30.14,115946.0,-44484.0,192570.0,51418.64,-39.96,34.0,X-MC,18.41,55.0,-0.38,1.43,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-4.42,-90.35,1187.35,24.20,349675.0,-275800.0,29450.0,3033.0,-85.87,36.0,X-SC,2.45,97.0,-0.79,0.22,23.34,X40N,NTT,FINANCE
54,QUESS,0.29,-34.38,117.68,42.84,50199.0,-22349.0,42657.0,424.0,-57.02,38.0,X-SC,17.11,80.0,-0.45,0.32,2.61,XY24,NTT,MISC
58,RELAXO,-1.23,-54.70,236.19,52.31,155323.0,-79398.0,65762.0,1176.0,-51.80,35.0,X-SC,14.80,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.60,-49.05,165.37,35.20,108413.0,-63112.0,65558.0,2096.0,-14.08,28.0,X-SC,22.33,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,-1.30,-4.26,29.60,24.08,35976.0,-5410.0,121540.0,1486.0,-4.88,51.0,X-SC,7.96,87.0,-0.15,0.90,11.32,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-0.88,-16.90,43.26,19.06,122291.0,-57478.0,282688.0,452.00,-54.31,40.0,X-LC,17.19,1.0,-0.47,2.09,2.36,X40,NTT,FMCG
73,TCS,-0.46,-24.46,63.60,23.58,207838.0,-105839.0,326790.0,4311.59,-34.45,31.0,X-LC,9.15,2.0,-0.51,2.42,2.40,X40,ATH,IT
43,INFY,1.16,-11.23,51.23,34.24,142292.0,-35136.0,277752.0,1972.00,-29.97,25.0,X-LC,8.69,3.0,-0.25,2.06,2.23,X40,NTT,IT
75,TMPV,-1.16,-14.00,55.04,33.33,154215.0,-45612.0,280188.0,600.00,-81.44,62.0,X-LC,3.94,4.0,-0.30,2.07,14.55,XY24,NTT,AUTO
34,HCLTECH,1.17,-9.83,37.33,23.83,81436.0,-23772.0,218152.0,1908.19,-5.29,30.0,X-LC,6.06,8.0,-0.29,1.62,4.66,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,-0.60,-29.21,87.72,32.89,47942.0,-22552.0,54654.0,1800.00,-357.00,64.0,L-MC,12.10,258.0,-0.47,0.40,50.51,XR,NTT,BANKS
67,SONACOMS,-1.80,-7.35,50.13,39.09,46982.0,-7438.0,93721.0,804.02,-26.55,60.0,M-MC,10.84,193.0,-0.16,0.69,32.58,AR,ATH,AUTO
19,COALINDIA,0.06,6.66,10.32,17.67,16562.0,10028.0,160488.0,478.51,29.12,58.0,L-LC,11.08,185.0,0.61,1.19,28.83,XY25,ATH,MINING
7,ATULAUTO,-1.05,-14.19,69.65,45.57,117813.0,-27982.0,169150.0,844.00,3667.57,58.0,M-SC,6.68,248.0,-0.24,1.25,28.01,XY24,NTT,AUTO
65,SIEMENS,2.24,-8.34,36.93,25.51,62995.0,-15515.0,170580.0,4671.50,43.36,66.0,H-LC,3.55,51.0,-0.25,1.26,25.70,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,-0.46,-52.32,205.56,45.70,173306.0,-92506.0,84309.0,26.40,-12.47,61.0,M-SC,18.19,204.0,-0.53,0.62,38.91,XY24,NTT,TRAVEL
1,ABB,-0.86,11.55,30.49,45.57,88982.0,30221.0,291840.0,7934.00,-28.35,66.0,H-MC,8.81,125.0,0.34,2.16,29.60,AR,NTT,ELECTRICAL
27,FINCABLES,5.15,16.31,78.75,107.91,135238.0,24087.0,171731.0,1641.55,4.52,75.0,M-SC,7.07,220.0,0.18,1.27,29.20,OX40N,ATH,CABLES
7,ATULAUTO,-1.05,-14.19,69.65,45.57,117813.0,-27982.0,169150.0,844.00,3667.57,58.0,M-SC,6.68,248.0,-0.24,1.25,28.01,XY24,NTT,AUTO
11,BANDHANBNK,-2.63,-12.83,120.75,92.42,293191.0,-35751.0,242808.0,400.00,108.33,70.0,H-SC,8.08,174.0,-0.12,1.80,33.76,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.75
1,25,45.79
2,50,77.86


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.90
MC    30.81
LC    25.26
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.51
X40      24.21
X40N     12.96
AR        9.35
XY25      9.12
XR        8.64
OX40N     7.33
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.20
H-SC    22.66
X-LC    21.77
M-SC    10.99
X-SC     9.53
H-MC     5.09
M-MC     2.12
H-LC     1.26
L-LC     1.19
M-LC     1.04
L-SC     0.72
L-MC     0.40
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.94,-14.64,53.42
IT,11.61,-39.82,109.90
FINANCE,11.20,-38.02,92.33
MISC,6.93,-38.91,95.73
ELECTRICAL,6.53,-6.32,45.12
PAINTS,5.14,-32.96,52.71
INSURANCE,4.95,-1.28,35.94
PHARMA,4.21,-3.20,35.69
AUTO,4.01,-14.92,58.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3412257.0,21
X40,1451947.0,15
X40N,1371987.0,10
XR,1324224.0,12
AR,1306342.0,10
OX40N,800985.0,10
XY25,385343.0,6
SR,303890.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3729225.0,24
X-SC,1519046.0,10
M-SC,1432678.0,14
X-MC,1380453.0,15
X-LC,1332723.0,12
H-MC,391099.0,3
L-SC,193640.0,2
M-LC,151939.0,1
M-MC,98673.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1259723.0      6
           AR         952533.0      5
           XR         849176.0      7
M-SC       XY24       839031.0      6
X-LC       X40        709595.0      6
X-SC       X40N       702677.0      4
           XY24       630070.0      4
X-MC       X40        556053.0      7
           X40N       439556.0      4
H-SC       OX40N      363903.0      4
M-SC       OX40N      338537.0      5
H-SC       SR         303890.0      2
X-LC       XY24       266007.0      2
           X40N       229754.0      2
H-MC       XY24       222305.0      1
X-MC       XY24       195121.0      2
           XY25       189723.0      2
X-SC       X40        186299.0      2
M-SC       XR         180072.0      2
H-MC       AR         168794.0      2
M-LC       XR         151939.0      1
X-LC       XY25       127367.0      2
L-SC       OX40N       98545.0      1
           XR          95095.0      1
M-SC       AR          75038.0      1
H-LC       AR          62995.0      1
M-MC       XY25        51691.0      1
L-MC       XR          47942.0      1
M-MC       AR          46982.0      1
L-LC       XY25        16562.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
